In [1]:
%pip install matplotlib kagglehub pandas openpyxl
%pip uninstall -y numpy
%pip install "numpy>=1.23.5,<2"
!python -m pip install protobuf==3.19.6 ml-dtypes==0.2.0
!python -m pip install tensorflow==2.10.0
!python -m pip list | findstr tensorflow

     ---------------------------------------- 0.0/250.9 kB ? eta -:--:--
     -------------- ------------------------ 92.2/250.9 kB 5.5 MB/s eta 0:00:01
     -------------------------------------- 250.9/250.9 kB 3.9 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


Found existing installation: numpy 1.26.4
Uninstalling numpy-1.26.4:
  Successfully uninstalled numpy-1.26.4
Note: you may need to restart the kernel to use updated packages.


  Using cached numpy-1.26.4-cp310-cp310-win_amd64.whl (15.8 MB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


tensorflow                   2.10.0
tensorflow-estimator         2.10.0
tensorflow-hub               0.16.1
tensorflow-io-gcs-filesystem 0.31.0



[notice] A new release of pip is available: 23.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("apithm/ecg-and-eeg-stress-features")

print("Path to dataset files:", path)

c:\Users\ommnj\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: C:\Users\ommnj\.cache\kagglehub\datasets\apithm\ecg-and-eeg-stress-features\versions\1


In [3]:
import tensorflow as tf
print(tf.__version__)
print(tf.config.list_physical_devices("GPU"))


2.10.0
[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [4]:
import os
import pandas as pd
ecg_path = os.path.join(path,"ECG (EO, AC1, AC2).xlsx")
ecg_eo = pd.read_excel(ecg_path, sheet_name="EO")
ecg_AC1 = pd.read_excel(ecg_path,sheet_name=("AC1"))
ecg_AC2 = pd.read_excel(ecg_path,sheet_name=("AC2"))

In [ ]:
#label the dataset
ecg_eo["label"] = 0
ecg_AC1["label"] = 1
ecg_AC2["label"] = 2

In [ ]:
#merge the 3 dataframes vertically to create a BIG dataset
ecg_df = pd.concat([ecg_eo,ecg_AC1,ecg_AC2],ignore_index=True)

In [8]:
#removing columns that willl do data leakage
ecg_df = ecg_df.drop(columns=["Subject NO."])